In [11]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt


In [12]:
train=pd.read_csv("C:/Users/HP/Desktop/train_emoji.csv",header=None)
test=pd.read_csv("C:/Users/HP/Desktop/test_emoji.csv",header=None)

In [13]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [14]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a very nice raise\t,2
3,she got me a nice present\t,2
4,ha ha ha it was so funny\t,2


In [15]:

# Creating dictionary for some emoji's, consisting of key - number and value - emoji 
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [16]:
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [17]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [18]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

D:\Users\HP\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\HP\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [21]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4], dtype=int64))

In [22]:
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]),
 array([ 3, 13, 15, 14,  6,  3,  2], dtype=int64))

In [23]:
f=open("C:/Users/HP/Desktop/glove.6B.50d.txt",encoding='utf-8')

In [24]:
embeddings_index = {}


for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [25]:
embeddings_index["i"].shape


(50,)

In [28]:
embeddings_index["eat"]

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01],
      dtype=float32)

In [29]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [30]:
spatial.distance.cosine(embeddings_index["india"], embeddings_index["paris"])

0.7530318051576614

In [31]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [32]:

print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [33]:
# training using rnn

In [41]:

# A simple RNN network to classify the emoji class from an input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
__________________________________________________

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 20ms/step - loss: 1.8688 - accuracy: 0.2045
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 1.7862 - accuracy: 0.2576
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 1.6525 - accuracy: 0.2576
Epoch 4/50
5/5 [==============================] - 0s 8ms/step - loss: 1.6461 - accuracy: 0.2424
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 1.4428 - accuracy: 0.3636
Epoch 6/50
5/5 [==============================] - 0s 7ms/step - loss: 1.5077 - accuracy: 0.3409
Epoch 7/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4398 - accuracy: 0.3636
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 1.3973 - accuracy: 0.4621
Epoch 9/50
5/5 [==============================] - 0s 8ms/step - loss: 1.3715 - accuracy: 0.4621
Epoch 10/50
5/5 [==============================] - 0s 8ms/step - loss: 1.3527 - accuracy: 0.4470
Epoch 11/50
5/5 [=====================

In [38]:
pred = model.predict_classes(embedding_matrix_test)


In [39]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.7142857142857143

In [40]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

3
['she', 'got', 'me', 'a', 'nice', 'present'] ❤ 😄
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 😄
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] 😄 😞
17
['Congratulation', 'for', 'having', 'a', 'baby'] 🍴 😄
21
['you', 'brighten', 'my', 'day'] 🍴 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
24
['Why', 'are', 'you', 'feeling', 'bad'] ❤ 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 🍴 ❤
34
['will', 'you', 'be', 'my', 'valentine'] ❤ 😄
35
['he', 'can', 'pitch', 'really', 'well'] 😄 ⚾
38
['See', 'you', 'at', 'the', 'restaurant'] ❤ 🍴
40
['I', 'will', 'run'] 😄 ⚾
42
['i', 'miss', 'her'] 😞 ❤
46
['What', 'you', 'did', 'was', 'awesome'] ⚾ 😄
51
['family', 'is', 'all', 'I', 'have'] 😄 ❤


In [42]:
# using lstm

In [43]:
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
________________________________________________

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:

# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 28ms/step - loss: 1.6004 - accuracy: 0.2197
Epoch 2/50
5/5 [==============================] - 0s 27ms/step - loss: 1.5282 - accuracy: 0.3561
Epoch 3/50
5/5 [==============================] - 0s 24ms/step - loss: 1.4519 - accuracy: 0.3561
Epoch 4/50
5/5 [==============================] - 0s 25ms/step - loss: 1.3842 - accuracy: 0.3939
Epoch 5/50
5/5 [==============================] - 0s 24ms/step - loss: 1.2850 - accuracy: 0.5379
Epoch 6/50
5/5 [==============================] - 0s 27ms/step - loss: 1.1321 - accuracy: 0.5833
Epoch 7/50
5/5 [==============================] - 0s 26ms/step - loss: 1.0576 - accuracy: 0.5909
Epoch 8/50
5/5 [==============================] - 0s 26ms/step - loss: 0.8625 - accuracy: 0.6818
Epoch 9/50
5/5 [==============================] - 0s 26ms/step - loss: 0.8297 - accuracy: 0.6894
Epoch 10/50
5/5 [==============================] - 0s 26ms/step - loss: 0.7217 - accuracy: 0.7348
Epoch 11/50
5/5 [============

In [46]:
pred = model.predict_classes(embedding_matrix_test)

In [47]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.8392857142857143

In [48]:

# Printing the sentences with the predicted and the labelled emoji
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

3
['she', 'got', 'me', 'a', 'nice', 'present'] ❤ 😄
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
16
['work', 'is', 'horrible'] 😄 😞
21
['you', 'brighten', 'my', 'day'] ❤ 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
32
['My', 'life', 'is', 'so', 'boring'] ❤ 😞
34
['will', 'you', 'be', 'my', 'valentine'] ❤ 😄
49
['go', 'away'] ⚾ 😞
